<a href="https://colab.research.google.com/github/lc0/practical-ml-dahoam/blob/safari/S_03_Production_use_tfjs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Production use

## Optional: check required files

Or upload your own model

In [30]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [8]:
!ls -laF /gdrive/'My Drive'/talks/2019-practical-ml/models

total 93658
-rw------- 1 root root    15208 May 27 22:04 model_hub.h5
-rw------- 1 root root 95890392 May 27 15:03 model_transfered_mobilenet.h5


In [0]:
!cp -R /gdrive/'My Drive'/talks/2019-practical-ml/models .

In [16]:
!ls -laF models/

total 93668
drwx------ 2 root root     4096 Oct 23 11:05 ./
drwxr-xr-x 1 root root     4096 Oct 23 11:05 ../
-rw------- 1 root root    15208 Oct 23 11:05 model_hub.h5
-rw------- 1 root root 95890392 Oct 23 11:05 model_transfered_mobilenet.h5


## TensorFlow.js

In [6]:
!pip install tensorflowjs

     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 17.3MB 7.8MB/s 
     |████████████████████████████████| 109.2MB 1.3MB/s 
     |████████████████████████████████| 81kB 27.8MB/s 
     |████████████████████████████████| 3.2MB 30.0MB/s 
     |████████████████████████████████| 491kB 47.5MB/s 
     |████████████████████████████████| 256kB 49.3MB/s 
     |████████████████████████████████| 890kB 35.4MB/s 
     |████████████████████████████████| 645kB 40.8MB/s 
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-cp36-none-any.whl size=32853 sha256=bb98f768e90ce87535c1be209b68a7719c21090a346331f8803f14606018bfb5
  Stored in directory: /root/.cache/pip/wheels/52/6c/b1/3e4b0e8daf42a92883c7641c0ea8ffb62e0490ebed2faa55ad
Successfully built PyInquirer
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.11.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is in

Convert model

In [0]:
# MODEL = 'model_hub.h5'
MODEL = 'model_transfered_mobilenet.h5'


In [0]:
!tensorflowjs_converter --input_format keras \
                        models/$MODEL \
                        prod/tfjs

In [20]:
!ls -laF prod/tfjs

total 31216
drwxr-xr-x 2 root root    4096 Oct 23 11:06 ./
drwxr-xr-x 3 root root    4096 Oct 23 11:06 ../
-rw-r--r-- 1 root root 4194304 Oct 23 11:06 group1-shard1of8.bin
-rw-r--r-- 1 root root 4194304 Oct 23 11:06 group1-shard2of8.bin
-rw-r--r-- 1 root root 4194304 Oct 23 11:06 group1-shard3of8.bin
-rw-r--r-- 1 root root 4194304 Oct 23 11:06 group1-shard4of8.bin
-rw-r--r-- 1 root root 4194304 Oct 23 11:06 group1-shard5of8.bin
-rw-r--r-- 1 root root 4194304 Oct 23 11:06 group1-shard6of8.bin
-rw-r--r-- 1 root root 4194304 Oct 23 11:06 group1-shard7of8.bin
-rw-r--r-- 1 root root 2586132 Oct 23 11:06 group1-shard8of8.bin
-rw-r--r-- 1 root root    4933 Oct 23 11:06 model.json


## App base

In [71]:
!wget https://github.com/lc0/practical-ml-dahoam/blob/master/prod/tfjs/shoe.png?raw=true -O prod/tfjs/shoe.png
!ls -laF prod/tfjs

--2019-10-23 11:20:44--  https://github.com/lc0/practical-ml-dahoam/blob/master/prod/tfjs/shoe.png?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/lc0/practical-ml-dahoam/raw/master/prod/tfjs/shoe.png [following]
--2019-10-23 11:20:44--  https://github.com/lc0/practical-ml-dahoam/raw/master/prod/tfjs/shoe.png
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lc0/practical-ml-dahoam/master/prod/tfjs/shoe.png [following]
--2019-10-23 11:20:45--  https://raw.githubusercontent.com/lc0/practical-ml-dahoam/master/prod/tfjs/shoe.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connecte

In [72]:
%%writefile prod/tfjs/index.html


<html>
<head>
    <!-- Load the latest version of TensorFlow.js -->
    <script src="https://unpkg.com/@tensorflow/tfjs"></script>
    <script src="https://unpkg.com/@tensorflow-models/mobilenet"></script>
    <script src="https://unpkg.com/@tensorflow-models/knn-classifier"></script>

</head>
<body>
    <style>
        .pred-container {
            margin-bottom: 20px;
        }
        .pred-container > div {
            display: inline-block;
            margin-right: 20px;
            vertical-align: top;
        }
        .row {
            display: table-row;
        }
        .cell {
            display: table-cell;
            padding-right: 20px;
        }
        #file-container {
            margin-bottom: 20px;
        }
        </style>
    <img id="img" crossOrigin src="shoe.png" width=192 height=192/>
    <div id="predictions"></div>


    <!-- Load index.js after the content of the page -->
    <script src="main.js"></script>
</body>
</html>

Overwriting prod/tfjs/index.html


In [53]:
%%writefile prod/tfjs/main.js



IMAGE_SIZE = 192
CLASSES = ['converse chuck 70', 'Exaggerated Soles', 'nike air max 90 essentials', 'nike react', 'Tie Dye sneakers']

const predictionsElement = document.getElementById('predictions');

async function app() {
  console.log('Loading model..');

  // Load the model.
  const model = await tf.loadLayersModel('/model.json');
  console.log('Sucessfully loaded model');

  // Make a prediction through the model on our image.
  const imgEl = document.getElementById('img');


  const result = await classify(model, imgEl);
  console.log(result);
}


async function classify(model, imgElement) {
  console.log('Predicting...');

  // The first start time includes the time it takes to extract the image
  // from the HTML and preprocess it, in additon to the predict() call.
  const startTime1 = performance.now();
  // The second start time excludes the extraction and preprocessing and
  // includes only the predict() call.
  let startTime2;
  const logits = tf.tidy(() => {
    // tf.browser.fromPixels() returns a Tensor from an image element.
    const img = tf.browser.fromPixels(imgElement, 3).toFloat();

    // Normalize the image from [0, 255] to [-1, 1].
    // const offset = tf.scalar(127.5);
    // const normalized = img.sub(offset).div(offset);
    normalized = img

    // Reshape to a single-element batch so we can pass it to predict.
    const batched = normalized.reshape([1, IMAGE_SIZE, IMAGE_SIZE, 3]);



    startTime2 = performance.now();
    // Make a prediction through mobilenet.
    return model.predict(batched);
  });

  console.log("Logits from model: ", logits)

  // Convert logits to probabilities and class names.
  classes = await getTopKClasses(logits, 2);
  console.log("Predicred classes: ", classes)

  const totalTime1 = performance.now() - startTime1;
  const totalTime2 = performance.now() - startTime2;
  console.log(`Done in ${Math.floor(totalTime1)} ms ` +
      `(not including preprocessing: ${Math.floor(totalTime2)} ms)`);

  showResults(imgElement, classes);
}

async function getTopKClasses(logits, topK) {
  const values = await logits.data();

  const valuesAndIndices = [];
  for (let i = 0; i < values.length; i++) {
    valuesAndIndices.push({value: values[i], index: i});
  }
  valuesAndIndices.sort((a, b) => {
    return b.value - a.value;
  });
  const topkValues = new Float32Array(topK);
  const topkIndices = new Int32Array(topK);
  for (let i = 0; i < topK; i++) {
    topkValues[i] = valuesAndIndices[i].value;
    topkIndices[i] = valuesAndIndices[i].index;
  }

  const topClassesAndProbs = [];
  for (let i = 0; i < topkIndices.length; i++) {
    topClassesAndProbs.push({
      className: CLASSES[topkIndices[i]],
      probability: topkValues[i]
    })
  }
  return topClassesAndProbs;
}

function showResults(imgElement, classes) {
  const predictionContainer = document.createElement('div');
  predictionContainer.className = 'pred-container';

  const imgContainer = document.createElement('div');
  imgContainer.appendChild(imgElement);
  predictionContainer.appendChild(imgContainer);

  const probsContainer = document.createElement('div');
  for (let i = 0; i < classes.length; i++) {
    const row = document.createElement('div');
    row.className = 'row';

    const classElement = document.createElement('div');
    classElement.className = 'cell';
    classElement.innerText = classes[i].className;
    row.appendChild(classElement);

    const probsElement = document.createElement('div');
    probsElement.className = 'cell';
    probsElement.innerText = classes[i].probability.toFixed(3);
    row.appendChild(probsElement);

    probsContainer.appendChild(row);
  }
  predictionContainer.appendChild(probsContainer);

  predictionsElement.insertBefore(
      predictionContainer, predictionsElement.firstChild);
}


app();

Overwriting prod/tfjs/main.js


### Serve our app directly from Colab



Serve files with python server module like 

```bash
python3 -m http.server
```

To see the result of work of the server we need `ngrock` for opening the port to outside

In [23]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


--2019-10-23 11:06:51--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.236.27.247, 3.218.65.25, 3.227.43.216, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.236.27.247|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  6.49MB/s    in 2.0s    

2019-10-23 11:06:54 (6.49 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


Now let's make `ngrok` to proxy the traffic from the localhost

In [0]:
get_ipython().system_raw('./ngrok http 8000 &')

In [74]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://ba90dce6.ngrok.io


In [75]:
!cd prod/tfjs/ && python3 -m http.server

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [23/Oct/2019 11:21:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:21:06] "GET /main.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:21:06] "GET /shoe.png HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:21:07] "GET /model.json HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:21:07] code 404, message File not found
127.0.0.1 - - [23/Oct/2019 11:21:07] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Oct/2019 11:21:08] "GET /group1-shard1of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:21:08] "GET /group1-shard2of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:21:08] "GET /group1-shard4of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:21:08] "GET /group1-shard5of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:21:08] "GET /group1-shard6of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:21:08] "GET /group1-shard3of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2019 11:21:11] "GET /group1-sha

Now you can open the `ngrok` URL and being able to see predictions

### Links


*  https://www.tensorflow.org/js/tutorials/conversion/import_keras
*  https://github.com/tensorflow/tfjs-examples/blob/master/mobilenet/index.js
* https://js.tensorflow.org/api/0.11.2/

